# 1. Введение

- В этом задании вам предстоит определить количество книг на фотографии.


Я в интернете нашел несколько датасетов, связанные с книгами, чтобы обучить модель предсказывать. Возможно, из-за того, что я не очень внимательно смотрел на датасеты, то будет дисбаланс.
Сами датасеты:
- https://universe.roboflow.com/valuable-object-detection/book-tjl7z
- https://universe.roboflow.com/books-mnb9z/books-7vad0
- https://universe.roboflow.com/book-mxasl/book-cduc4
- https://universe.roboflow.com/books-3jjp1/book-detection-aafpi


# 2. Установка зависимостей

In [14]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.3 MB/s eta 0:00:00


Я выбрал YOLOv8s по нескольким причинам:
- Простота и готовое решение — модель уже реализована и активно поддерживается, поэтому нет смысла изобретать велосипед и писать собственную архитектуру с нуля.
- Скорость работы — YOLOv8s достаточно лёгкая, хорошо подходит для обучения и инференса даже в Colab без серьёзных требований к железу.
- Поддержка датасетов — многие открытые датасеты для задач детекции книг уже размечены под формат YOLOv8, что сильно ускоряет подготовку данных и запуск обучения.

# 3. Обработка датасетов


In [28]:
import os, zipfile, random, shutil, glob

# Пути к архивам
zips = [
    "/content/Book Detection.v1i.yolov8.zip",
    "/content/Book.v1i.yolov8.zip",
    "/content/books.v1-roboflow-instant-1--eval-.yolov8.zip",
    "/content/book.v1i.yolov8 (1).zip"
]

datasets_dir = "/content/datasets"
os.makedirs(datasets_dir, exist_ok=True)

# Разархивируем каждый датасет
extracted_dirs = []
for zip_path in zips:
    folder_name = os.path.splitext(os.path.basename(zip_path))[0]
    extract_path = os.path.join(datasets_dir, folder_name)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    extracted_dirs.append(extract_path)

# Начинаем собирать датасеты в одно место
combined_dir = os.path.join(datasets_dir, "books_combined_all")
for split in ["train", "valid", "test"]:
    for sub in ["images", "labels"]:
        os.makedirs(os.path.join(combined_dir, split, sub), exist_ok=True)

# Функция для копирования всех картинок и аннотаций
def copy_all_data(src_dir, split):
    src_img_dir = os.path.join(src_dir, split, "images")
    src_lbl_dir = os.path.join(src_dir, split, "labels")
    dst_img_dir = os.path.join(combined_dir, split, "images")
    dst_lbl_dir = os.path.join(combined_dir, split, "labels")
    for img_path in glob.glob(os.path.join(src_img_dir, "*.*")):
        filename = os.path.basename(img_path)
        stem, _ = os.path.splitext(filename)
        label_path = os.path.join(src_lbl_dir, stem + ".txt")
        shutil.copy(img_path, os.path.join(dst_img_dir, filename))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dst_lbl_dir, stem + ".txt"))

# Копируем данные из всех датасетов
for d in extracted_dirs:
    for split in ["train", "valid", "test"]:
        copy_all_data(d, split)

print("Все изображения и разметка объединены")


Все изображения и разметка объединены


Здесь я загрузил датасеты вручную в соотвествующую директорию. Далее объединил эти датасеты, чтобы обучить модель.


In [29]:
yaml_path = os.path.join(combined_dir, "data.yaml")

yaml_content = f"""
train: {combined_dir}/train/images
val: {combined_dir}/valid/images
test: {combined_dir}/test/images

nc: 1
names: ["book"]
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("data.yaml создан:", yaml_path)


data.yaml создан: /content/datasets/books_combined_all/data.yaml


Создал data.yml для модели, чтобы она могла обучаться

# 4. Обучение модели

In [ ]:
!yolo task=detect mode=train \
    model=yolov8s.pt \
    data=/content/datasets/books_combined_all/data.yaml \
    epochs=20 \
    imgsz=1020  \
    batch=16 \
    device=0 \
    workers=8 \
    augment=True \
    cache=True


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/books_combined_all/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1020, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plo

Здесь происходит обучение модели.

# 4. Формирование ответов

In [27]:
import pandas as pd
from ultralytics import YOLO

# Разархивируем тестовые изображения
zip_path = "/content/task_images.zip"
extract_dir = "/content/task_images"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Картинки лежат внутри task_images/task_images/
src_dir = os.path.join(extract_dir, "task_images")
test_dir = "/content/test_images"
os.makedirs(test_dir, exist_ok=True)

import shutil
for img_path in glob.glob(os.path.join(src_dir, "*.*")):
    shutil.copy(img_path, test_dir)

print(f"Все {len(os.listdir(test_dir))} изображений готовы для инференса")

# Загружаем обученную модель
model = YOLO("/content/runs/detect/train3/weights/best.pt") # Здесь надо указать путь на последнюю обученную модель

# Подсчёт книг на всех изображениях
image_paths = glob.glob(os.path.join(test_dir, "*.*"))
results_list = []

for img_path in image_paths:
    result = model.predict(img_path, imgsz=1020, conf=0.3) # потому что картинки больше, чем в обучающей выборке
    n_books = len(result[0].boxes)  # количество боксов
    image_id = os.path.basename(img_path)
    results_list.append([image_id, n_books])

# Сохраняем в CSV
submission_df = pd.DataFrame(results_list, columns=['image_id', 'number_of_books'])
submission_df.to_csv('solution.csv', index=False)

print("solution.csv создан")


Все 1148 изображений готовы для инференса

image 1/1 /content/test_images/152.jpg: 960x736 2 books, 26.1ms
Speed: 6.0ms preprocess, 26.1ms inference, 2.4ms postprocess per image at shape (1, 3, 960, 736)

image 1/1 /content/test_images/38.jpg: 736x960 2 books, 26.2ms
Speed: 6.8ms preprocess, 26.2ms inference, 6.4ms postprocess per image at shape (1, 3, 736, 960)

image 1/1 /content/test_images/1078.jpg: 736x960 4 books, 25.0ms
Speed: 9.4ms preprocess, 25.0ms inference, 4.6ms postprocess per image at shape (1, 3, 736, 960)

image 1/1 /content/test_images/0.jpg: 960x736 1 book, 26.3ms
Speed: 4.3ms preprocess, 26.3ms inference, 1.9ms postprocess per image at shape (1, 3, 960, 736)

image 1/1 /content/test_images/782.jpg: 960x736 1 book, 24.8ms
Speed: 3.6ms preprocess, 24.8ms inference, 1.9ms postprocess per image at shape (1, 3, 960, 736)

image 1/1 /content/test_images/939.jpg: 960x736 1 book, 24.8ms
Speed: 3.6ms preprocess, 24.8ms inference, 1.9ms postprocess per image at shape (1, 3, 9

Загрузил task_images.zip, также как и другие датасеты и предобработал пути. Подсчитал количества книг для каждого image_id. Сформировал solution.csv.

# Выводы

Скорее всего низкая метрика связана с плохим сбором датасета, а также недостаточным хорошим обучением самой модели. Возможно, надо было показать моделе еще датасет с "некнигами", чтобы она могла более уверенно отличать их. Также, скорее всего, не хватило итераций для улучшения результата.